# Convert SAM stat output to DEPHY standards for SEA STARR

#### Code modified by Ehsan Erfani for UW-SAM, June 2022 @UW
#### Code created by Michael Diamond for NOAA-SAM, June 2022 @NOAA Boulder

In [1]:
#Import libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import datetime
#import nerodia as nero

### Constants

In [2]:
year  = str(datetime.datetime.now().year)
month = str(datetime.datetime.now().month)
day   = str(datetime.datetime.now().day)

#Constants from Romps (2017)
Ttrip = 273.16 # K
ptrip = 611.65 # Pa
E0v = 2.3740e6 # J/kg
E0i = 0.3337e6 # J/kg
g = 9.81 # m/s2
Rd = 287.04 # J/kg/K
Rv = 461. # J/kg/K
cvd = 719. # J/kg/K
cvv = 1418. # J/kg/K
cvl = 4119. # J/kg/K
cvi = 1861. # J/kg/K
cpd = cvd + Rd # J/kg/K
cpv = cvv + Rv # J/kg/K

#Constants from Bolton (1980)
Lv = 2.501e6 #J/kg

### Converts SAM output to DEPHY standards

In [3]:
def create_DEPHY_output(input_path,output_path,caseid):
    """
    Convert SAM stat output to DEPHY/CF standards for SEA STARR.

    Parameters:
    -----------
    input_path : str
    Path to input data

    output_path : str
    Path in which to save output data

    caseid : str
    Case name
    """

    #Load data
    inp = xr.open_dataset(input_path)
    out = xr.Dataset()


    #=============
    ###Coordinates
    #=============

    #Time
    out['time'] = np.array((inp.time.values-226)*24*3600,dtype=int)
    out['time'].attrs = {'units' : 'seconds since 2017-08-14 00:00:00', 'standard_name' : 'time', 'calendar' : 'Gregorian'}

    #Height
    out['zf'] = inp.z.values
    out['zf'].attrs = {'units' : 'm', 'standard_name' : 'height', 'comment' : 'altitude of mid-level points above sea surface'}


    #=============================
    ###Dynamics and thermodynamics
    #=============================

    #Pressure
    out['pa'] = (['time','zf'],inp['PRES'].values*100)
    out['pa'].attrs = {'units' : 'Pa', 'standard_name' : 'air_pressure'}

    #Density
    out['rho'] = (['time','zf'],inp['RHO'].values)
    out['rho'].attrs = {'units' : 'kg m-3', 'standard_name' : 'air_volumic_mass'}

    #Temperature
    out['ta'] = (['time','zf'],inp['TABS'].values)
    out['ta'].attrs = {'units' : 'K', 'standard_name' : 'air_temperature'}

    #Specific humidity
    out['qv'] = (['time','zf'],inp['QV'].values/1000)
    out['qv'].attrs = {'units' : '1', 'standard_name' : 'specific_humidity'}

    #Total water
    out['qt'] = (['time','zf'],(inp['QV']+inp['QC']+inp['QR']).values/1000)
    out['qt'].attrs = {'units' : '1', 'standard_name' : 'total_water_content', 'comment' : 'specific mass'}

    #Relative humidity
    out['hur'] = (['time','zf'],(inp['RELH']).values/100)
    out['hur'].attrs = {'units' : '1', 'standard_name' : 'relative_humidity', 'comment' : 'relative to liquid'}

    #Zonal wind
    out['ua'] = (['time','zf'],(inp['U']).values)
    out['ua'].attrs = {'units' : 'm s-1', 'standard_name' : 'eastward_wind'}

    #Meridional wind
    out['va'] = (['time','zf'],(inp['V']).values)
    out['va'].attrs = {'units' : 'm s-1', 'standard_name' : 'northward_wind'}

    #Vertical velocity
    out['wa'] = (['time','zf'],(inp['WOBS']).values)
    out['wa'].attrs = {'units' : 'm s-1', 'standard_name' : 'upward_air_velocity'}

    #Potential temperature
    out['theta'] = (['time','zf'],(inp['THETA']).values)
    out['theta'].attrs = {'units' : 'K', 'standard_name' : 'air_potential_temperature'}

    #Liquid water potential temperature
    out['thetal'] = (['time','zf'],(inp['THETAL']).values)
    out['thetal'].attrs = {'units' : 'K', 'standard_name' : 'air_liquid_potential_temperature'}

    #TKE
    out['tke'] = (['time','zf'],(inp['TKE']+inp['TKES']).values)
    out['tke'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'specific_turbulent_kinetic_energy', 'comment' : 'sum of resolved and SGS'}

    #Sensible heat flux
    out['hfss'] = (['time'],(inp['SHF']).values)
    out['hfss'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upward_sensible_heat_flux', 'comment' : 'postive upward'}

    #Latent heat flux
    out['hfls'] = (['time'],(inp['LHF']).values)
    out['hfls'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upward_latent_heat_flux', 'comment' : 'positive upward'}

    #Surface pressure
    out['ps'] = (['time'],(inp['Ps']).values*100)
    out['ps'].attrs = {'units' : 'Pa', 'standard_name' : 'surface_pressure'}

    #Surface temperature
    out['ts'] = (['time'],(inp['SST']).values)
    out['ts'].attrs = {'units' : 'K', 'standard_name' : 'surface_temperature'}

    #Inversion height
    out['zi'] = (['time'],(inp['ZINV']).values*1000)
    out['zi'].attrs = {'units' : 'm', 'standard_name' : 'inversion_height'}

    #Variance of inversion height
    out['zivar'] = (['time'],(inp['ZINV2']).values*(1000)**2)
    out['zivar'].attrs = {'units' : 'm2', 'standard_name' : 'variance_of_inversion_height'}

    #Variance of vertical velocity
    out['w2'] = (['time','zf'],(inp['W2']).values)
    out['w2'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'variance_of_upward_air_velocity'}

    #Variance of zonal velocity
    out['u2'] = (['time','zf'],(inp['U2']).values)
    out['u2'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'variance_of_eastward_wind'}

    #Variance of meridional velocity
    out['v2'] = (['time','zf'],(inp['V2']).values)
    out['v2'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'variance_of_northward_wind'}

    #Vertical velocity skewness
    out['wskew'] = (['time','zf'],(inp['WSKEW']).values)
    out['wskew'].attrs = {'units' : '1', 'standard_name' : 'skewness_of_upward_air_velocity'}

    #Total water flux
    out['wqt'] = (['time','zf'],(inp['QTFLUX']).values/Lv/inp['RHO'].values)
    out['wqt'].attrs = {'units' : 'm s-1', 'standard_name' : 'flux_of_total_water_content'}

    #Temperature flux
    out['wthl'] = (['time','zf'],(inp['TLFLUX']).values/cpd/inp['RHO'].values)
    out['wthl'].attrs = {'units' : 'K m s-1', 'standard_name' : 'flux_of_air_liquid_water_potential_temperature'}

    #Zonal momentum flux
    out['wu'] = (['time','zf'],(inp['UW']).values)
    out['wu'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'flux_of_eastward_momentum'}

    #Meridional momentum flux
    out['wv'] = (['time','zf'],(inp['VW']).values)
    out['wv'].attrs = {'units' : 'm2 s-2', 'standard_name' : 'flux_of_northward_momentum'}

    #CAPE
    out['cape'] = (['time'],(inp['CAPE']).values)
    out['cape'].attrs = {'units' : 'J kg-1', 'standard_name' : 'atmosphere_convective_available_potential_energy'}


    #====================================
    ###Aerosol, clouds, and precipitation
    #====================================

    #Cloud water mixing ratio
    out['ql'] = (['time','zf'],(inp['QC']).values/1000)
    out['ql'].attrs = {'units' : '1', 'standard_name' : 'mass_fraction_of_cloud_liquid_water_in_air', 'comment' : 'specific mass'}

    #Rain water mixing ratio
    out['qr'] = (['time','zf'],(inp['QR']).values/1000)
    out['qr'].attrs = {'units' : '1', 'standard_name' : 'mass_fraction_of_rain_water_in_air', 'comment' : 'specific mass'}

    #Cloud fraction in air
    out['cl'] = (['time','zf'],(inp['CLD']).values)  # NOAA-SAM variable name: cl
    out['cl'].attrs = {'units' : '1', 'standard_name' : 'cloud_area_fraction_in_atmospheric_layer', 'comment' : 'liquid water threshold of 0.01 g/kg'}

    #Cloud fraction
    out['clt'] = (['time'],(inp['ISCCPTOT']).values)  # NOAA-SAM variable name: AFOPD2 
    out['clt'].attrs = {'units' : '1', 'standard_name' : 'cloud_area_fraction', 'comment' : 'fraction of atmospheric columns with cloud optical thickness > 2'}

    #Precipitation flux at surface
    out['pr'] = (['time'],(inp['PREC']).values/(24*3600))
    out['pr'].attrs = {'units' : 'kg m-2 s-1', 'standard_name' : 'precipitation_flux_at_surface'}

    #Precipitation flux in air
    out['prf'] = (['time','zf'],(inp['PRECIP']).values/(24*3600))
    out['prf'].attrs = {'units' : 'kg m-2 s-2', 'standard_name' : 'precipitation_flux_in_air'}

    #CWP
    out['cwp'] = (['time'],(inp['CWP']).values/1000)
    out['cwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_cloud_water', 'comment' : 'scene; cloud water only'}

    #LWP
    out['lwp'] = (['time'],(inp['CWP']+inp['RWP']).values/1000)
    out['lwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_liquid_water', 'comment' : 'scene; cloud and rain'}

    #RWP
    out['rwp'] = (['time'],(inp['RWP']).values/1000)
    out['rwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_rain_water', 'comment' : 'scene'}

    #Total aerosol number
    #out['nt'] = (['time','zf'],(inp['NT']/inp['RHO']).values/1e6)  # NOAA SAM variable
    out['nt'] = (['time','zf'],(inp['NAd']+inp['NC']+inp['NR']).values/1e-6)
    out['nt'].attrs = {'units' : 'kg-1', 'standard_name' : 'number_of_total_aerosol_in_air'}

    #Dry aerosol number
    #out['na'] = (['time','zf'],(inp['NA']/inp['RHO']).values/1e6)  # NOAA SAM variable
    out['na'] = (['time','zf'],(inp['NAd']).values/1e-6)
    out['na'].attrs = {'units' : 'kg-1', 'standard_name' : 'number_of_dry_aerosol_in_air'}

    #Cloud droplet number
    #out['nc'] = (['time','zf'],(inp['NC']/inp['RHO']).values/1e6)  # NOAA SAM variable
    out['nc'] = (['time','zf'],(inp['NC']).values/1e-6)
    out['nc'].attrs = {'units' : 'kg-1', 'standard_name' : 'number_of_cloud_droplets_in_air'}

    #Rain droplet number
    #out['nr'] = (['time','zf'],(inp['NR']/inp['RHO']).values/1e6)  # NOAA SAM variable
    out['nr'] = (['time','zf'],(inp['NR']).values/1e-6)
    out['nr'].attrs = {'units' : 'kg-1', 'standard_name' : 'number_of_rain_droplets_in_air'}

    #Effective radius
    #out['cer'] = (['time','zf'],(inp['cer']).values)  # NOAA SAM variable
    out['cer'] = (['time','zf'],(inp['QC']/inp['QCOEFFR']).values*1e-6)
    out['cer'].attrs = {'units' : 'm', 'standard_name' : 'effective_radius_of_cloud_droplets'}

    # Currently, These variables are not in outputs of UW SAM
    # #Collection rate of cloud droplets
    # out['crnc'] = (['time','zf'],(inp['NCCOL']/inp['RHO']).values/1e6/24/3600)
    # out['crnc'].attrs = {'units' : 'kg-1 s-1', 'standard_name' : 'collection_rate_of_cloud_droplet_number'}

    # #Collection rate of rain droplets
    # out['crnr'] = (['time','zf'],(inp['NRCOL']/inp['RHO']).values/1e6/24/3600)
    # out['crnr'].attrs = {'units' : 'kg-1 s-1', 'standard_name' : 'collection_rate_of_rain_droplet_number'}

    # #Collection rate of cloud water
    # out['crql'] = (['time','zf'],(inp['QCCOL']).values/1000/24/3600)
    # out['crql'].attrs = {'units' : 's-1', 'standard_name' : 'collection_rate_of_cloud_water_mass'}

    # #Collection rate of rain water
    # out['crqr'] = (['time','zf'],(inp['QRCOL']).values/1000/24/3600)
    # out['crqr'].attrs = {'units' : 's-1', 'standard_name' : 'collection_rate_of_rain_water_mass'}

    # #Condensation rate
    # out['cond'] = (['time','zf'],(inp['QVSNK']).values/1000/24/3600)
    # out['cond'].attrs = {'units' : 's-1', 'standard_name' : 'rate_of_condensation'}

    # #Evaporation rate
    # out['evap'] = (['time','zf'],(inp['QVSRC']).values/1000/24/3600)
    # out['evap'].attrs = {'units' : 's-1', 'standard_name' : 'rate_of_evaporation'}


    #===========
    ###Radiation
    #===========

    #LW heating rate
    out['rlh'] = (['time','zf'],(inp['RADQRLW']).values*cpd/24/3600)
    out['rlh'].attrs = {'units' : 'W kg-1', 'standard_name' : 'longwave_heating_rate_in_air'}

    #SW heating rate
    out['rsh'] = (['time','zf'],(inp['RADQRSW']).values*cpd/24/3600)
    out['rsh'].attrs = {'units' : 'W kg-1', 'standard_name' : 'shortwave_heating_rate_in_air'}

    #Clear-sky LW heating rate
    out['rlhcs'] = (['time','zf'],(inp['RADQRCLW']).values*cpd/24/3600)
    out['rlhcs'].attrs = {'units' : 'W kg-1', 'standard_name' : 'longwave_heating_rate_in_air_assuming_clear_sky'}

    #Clear-sky SW heating rate
    out['rshcs'] = (['time','zf'],(inp['RADQRCSW']).values*cpd/24/3600)
    out['rshcs'].attrs = {'units' : 'W kg-1', 'standard_name' : 'shortwave_heating_rate_in_air_assuming_clear_sky'}

    #Downwelling LW flux
    out['rld'] = (['time','zf'],(inp['RADLWDN']).values)
    out['rld'].attrs = {'units' : 'W m-2', 'standard_name' : 'downwelling_longwave_flux_in_air'}

    #Upwelling LW flux
    out['rlu'] = (['time','zf'],(inp['RADLWUP']).values)
    out['rlu'].attrs = {'units' : 'W m-2', 'standard_name' : 'upwelling_longwave_flux_in_air'}

    #Downwelling SW flux
    out['rsd'] = (['time','zf'],(inp['RADSWDN']).values)
    out['rsd'].attrs = {'units' : 'W m-2', 'standard_name' : 'downwelling_shortwave_flux_in_air'}

    #Upwelling SW flux
    out['rsu'] = (['time','zf'],(inp['RADSWUP']).values)
    out['rsu'].attrs = {'units' : 'W m-2', 'standard_name' : 'upwelling_shortwave_flux_in_air'}

    #TOA incoming SW flux
    out['rsdt'] = (['time'],(inp['SOLIN']).values)
    out['rsdt'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_incoming_shortwave_flux'}

    #TOA outgoing SW flux
    out['rsut'] = (['time'],(inp['SOLIN']-inp['SWNTOA']).values)
    out['rsut'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_shortwave_flux'}

    #TOA outgoing LW flux
    out['rlut'] = (['time'],(inp['LWNTOA']).values)
    out['rlut'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_longwave_flux'}

    #TOA outgoing clear-sky SW flux
    out['rsutcs'] = (['time'],(inp['SOLIN']-inp['SWNTOAC']).values)
    out['rsutcs'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_shortwave_flux_assuming_clear_sky'}

    #TOA outgoing clear-sky LW flux
    out['rlutcs'] = (['time'],(inp['LWNTOAC']).values)
    out['rlutcs'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_longwave_flux_assuming_clear_sky'}

    #Surface downwelling LW
    out['rlds'] = (['time'],(inp['LWDS']).values)
    out['rlds'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_downwelling_longwave_flux'}

    #Surface upwelling LW
    out['rlds'] = (['time'],(inp['LWNS']+inp['LWDS']).values)
    out['rlds'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upwelling_longwave_flux'}

    #Surface downwelling SW
    out['rsds'] = (['time'],(inp['SWDS']).values)
    out['rsds'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_downwelling_shortwave_flux'}

    #Surface upwelling SW
    out['rsus'] = (['time'],(inp['SWNS']+inp['SWDS']).values)
    out['rsus'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upwelling_shortwave_flux'}


    #==========
    ###Metadata
    #==========

    out.attrs['case'] = 'SEA_STARR/'+caseid
    out.attrs['title'] = 'Statistical output for SEA_STARR/%s case' % caseid
    out.attrs['reference'] = 'Diamond et al. (in prep), ACPD'
    out.attrs['authors'] = 'Michael Diamond (michael.diamond@noaa.gov) and Ehsan Erfani (Erfani@uw.edu)'
    out.attrs['version'] = 'Created on ' + year + '-' + month + '-' + day
    out.attrs['format_version'] = 'DEPHY SCM format version 1'
    out.attrs['modifications'] = ''
    out.attrs['script'] = 'DEPHY_stat.ipynb'


    #===========
    ###Save file
    #===========

    os.system('rm %s' % output_path) #Clear old file if it exists
    out.to_netcdf(output_path)

In [4]:
input_path  = '../OUT_STAT/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_CTRL_LD.nc'
output_path = './SEA_STARR_CTRL_STAT_SAM_UW.nc'
caseid      = 'CTRL'

create_DEPHY_output(input_path,output_path,caseid)

In [5]:
input_path  = '../OUT_STAT/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_NA100_LD.nc'
output_path = './SEA_STARR_NA100_STAT_SAM_UW.nc'
caseid      = 'NA100'

create_DEPHY_output(input_path,output_path,caseid)

In [4]:
input_path  = '../OUT_STAT/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_NA30_LD.nc'
output_path = './SEA_STARR_NA30_STAT_SAM_UW.nc'
caseid      = 'NA30'

create_DEPHY_output(input_path,output_path,caseid)